In [1]:
#%pip install cython
#%pip install pyranges

In [2]:
import pandas as pd
import numpy as np
import pyranges as pr

# Load proteins, mutations, domains and regions tables

## Preprocessing entrada_dbs table

In [150]:
# proteins from each LLPS database with their roles and mlos
entrada_dbs = pd.read_csv('entrada_dbs.tsv.txt', sep='\t').rename(columns={'uniprot': 'uniprot_acc'})

In [151]:
entrada_dbs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405 entries, 0 to 8404
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniprot_acc  8389 non-null   object
 1   organism     8405 non-null   object
 2   mlo          8398 non-null   object
 3   rol          8405 non-null   object
 4   db           8405 non-null   object
dtypes: object(5)
memory usage: 328.4+ KB


In [152]:
# There's NaNs uniprots
entrada_dbs[entrada_dbs.uniprot_acc.isnull()]

,uniprot_acc,organism,mlo,rol,db
5688,NaN,Homo sapiens,Centrosome/Spindle pole body,client,drllps
6105,NaN,Homo sapiens,Nucleolus,client,drllps
7417,NaN,Homo sapiens,Nucleolus,client,drllps
7664,NaN,Homo sapiens,Nucleolus,client,drllps
7747,NaN,Homo sapiens,Nucleolus,client,drllps
7751,NaN,Homo sapiens,Postsynaptic density,client,drllps
7889,NaN,Homo sapiens,Stress granule,client,drllps
7914,NaN,Homo sapiens,Postsynaptic density,client,drllps
7943,NaN,Homo sapiens,Postsynaptic density,client,drllps
8069,NaN,Homo sapiens,Postsynaptic density,client,drllps


In [153]:
# Drop these rows
entrada_dbs = entrada_dbs[entrada_dbs.uniprot_acc.notnull()]
entrada_dbs

,uniprot_acc,organism,mlo,rol,db
0,P35637,Homo sapiens,cytoplasmic stress granule,driver,phasepro
1,P35637,Homo sapiens,cytoplasmic ribonucleoprotein granule,driver,phasepro
2,Q06787,Homo sapiens,cytoplasmic stress granule,driver,phasepro
3,Q06787,Homo sapiens,cytoplasmic ribonucleoprotein granule,driver,phasepro
4,Q06787,Homo sapiens,"synaptosome, neuron projection",driver,phasepro
...,...,...,...,...,...
8398,O95670,Homo sapiens,Nucleolus,client,drllps
8399,O95670,Homo sapiens,Postsynaptic density,client,drllps
8401,Q9H269,Homo sapiens,Postsynaptic density,client,drllps
8402,Q9Y3D7,Homo sapiens,Postsynaptic density,client,drllps


In [154]:
# There's NaNs mlos
entrada_dbs[entrada_dbs.mlo.isnull()]

,uniprot_acc,organism,mlo,rol,db
7,Q92804,Homo sapiens,NaN,driver,phasepro
10,Q13148,Homo sapiens,NaN,driver,phasepro
12,Q01844,Homo sapiens,NaN,driver,phasepro
53,O43561,Homo sapiens,NaN,driver,phasepro
56,P62993,Homo sapiens,NaN,driver,phasepro
59,Q07889,Homo sapiens,NaN,driver,phasepro
114,P43243,Homo sapiens,NaN,driver,phasepro


In [155]:
missing_mlo = entrada_dbs[entrada_dbs.mlo.isnull()]

In [156]:
# Search these uniprot entries in PhaSePro dataset
phasepro_human = pd.read_csv('../PhaSePro_human.txt', sep='\t', header= None)
phasepro_human.shape

phasepro_missing = phasepro_human[phasepro_human[2].isin(missing_mlo.uniprot_acc)].iloc[:, [2,14]]
phasepro_missing.rename(columns={2: 'uniprot_acc', 14: 'mlo'}, inplace= True)
phasepro_missing.mlo = phasepro_missing.mlo.str.strip().str.strip(';')              # strip blank space then ;
phasepro_missing['db'] = 'phasepro'
phasepro_missing

,uniprot_acc,mlo,db
2,Q92804,nuclear protein granule,phasepro
3,Q13148,cytoplasmic stress granule; cytoplasmic ribonu...,phasepro
4,Q01844,nuclear protein granule,phasepro
24,O43561,TCR signalosome; LAT signalosome,phasepro
25,P62993,TCR signalosome; LAT signalosome,phasepro
26,Q07889,TCR signalosome; LAT signalosome,phasepro
51,P43243,nuclear protein granule,phasepro


In [157]:
# Set mlo col as list-like and explode() to separate list elements into separate rows
phasepro_missing = phasepro_missing.assign(mlo= phasepro_missing.mlo.str.split(';')).explode('mlo') # explode() is used to transform each element of a list-like to a row
phasepro_missing.mlo = phasepro_missing.mlo.str.strip()
phasepro_missing

,uniprot_acc,mlo,db
2,Q92804,nuclear protein granule,phasepro
3,Q13148,cytoplasmic stress granule,phasepro
3,Q13148,cytoplasmic ribonucleoprotein granule,phasepro
4,Q01844,nuclear protein granule,phasepro
24,O43561,TCR signalosome,phasepro
24,O43561,LAT signalosome,phasepro
25,P62993,TCR signalosome,phasepro
25,P62993,LAT signalosome,phasepro
26,Q07889,TCR signalosome,phasepro
26,Q07889,LAT signalosome,phasepro


In [158]:
to_add = missing_mlo.merge(phasepro_missing, on=['uniprot_acc', 'db']).drop(columns='mlo_x').rename(columns= {'mlo_y': 'mlo'})
to_add

,uniprot_acc,organism,rol,db,mlo
0,Q92804,Homo sapiens,driver,phasepro,nuclear protein granule
1,Q13148,Homo sapiens,driver,phasepro,cytoplasmic stress granule
2,Q13148,Homo sapiens,driver,phasepro,cytoplasmic ribonucleoprotein granule
3,Q01844,Homo sapiens,driver,phasepro,nuclear protein granule
4,O43561,Homo sapiens,driver,phasepro,TCR signalosome
5,O43561,Homo sapiens,driver,phasepro,LAT signalosome
6,P62993,Homo sapiens,driver,phasepro,TCR signalosome
7,P62993,Homo sapiens,driver,phasepro,LAT signalosome
8,Q07889,Homo sapiens,driver,phasepro,TCR signalosome
9,Q07889,Homo sapiens,driver,phasepro,LAT signalosome


In [159]:
# Drop those missing mlos entries
entrada_dbs = entrada_dbs.drop(missing_mlo.index)
# And concatenate the subset from phasepro (to_add)
entrada_dbs = pd.concat([entrada_dbs, to_add], ignore_index= True)

In [160]:
entrada_dbs

,uniprot_acc,organism,mlo,rol,db
0,P35637,Homo sapiens,cytoplasmic stress granule,driver,phasepro
1,P35637,Homo sapiens,cytoplasmic ribonucleoprotein granule,driver,phasepro
2,Q06787,Homo sapiens,cytoplasmic stress granule,driver,phasepro
3,Q06787,Homo sapiens,cytoplasmic ribonucleoprotein granule,driver,phasepro
4,Q06787,Homo sapiens,"synaptosome, neuron projection",driver,phasepro
...,...,...,...,...,...
8388,P62993,Homo sapiens,TCR signalosome,driver,phasepro
8389,P62993,Homo sapiens,LAT signalosome,driver,phasepro
8390,Q07889,Homo sapiens,TCR signalosome,driver,phasepro
8391,Q07889,Homo sapiens,LAT signalosome,driver,phasepro


In [161]:
entrada_dbs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8393 entries, 0 to 8392
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniprot_acc  8393 non-null   object
 1   organism     8393 non-null   object
 2   mlo          8393 non-null   object
 3   rol          8393 non-null   object
 4   db           8393 non-null   object
dtypes: object(5)
memory usage: 328.0+ KB


In [166]:
# Check for duplicated rows
entrada_dbs.duplicated().sum()

127

In [168]:
entrada_dbs.drop_duplicates(inplace= True)

In [169]:
entrada_dbs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8266 entries, 0 to 8391
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniprot_acc  8266 non-null   object
 1   organism     8266 non-null   object
 2   mlo          8266 non-null   object
 3   rol          8266 non-null   object
 4   db           8266 non-null   object
dtypes: object(5)
memory usage: 387.5+ KB


In [171]:
entrada_dbs.drop(columns='organism', inplace= True)

In [172]:
entrada_dbs

,uniprot_acc,mlo,rol,db
0,P35637,cytoplasmic stress granule,driver,phasepro
1,P35637,cytoplasmic ribonucleoprotein granule,driver,phasepro
2,Q06787,cytoplasmic stress granule,driver,phasepro
3,Q06787,cytoplasmic ribonucleoprotein granule,driver,phasepro
4,Q06787,"synaptosome, neuron projection",driver,phasepro
...,...,...,...,...
8381,P33778,Nucleolus,client,drllps
8384,Q13148,cytoplasmic ribonucleoprotein granule,driver,phasepro
8387,O43561,LAT signalosome,driver,phasepro
8389,P62993,LAT signalosome,driver,phasepro


## Load the others tables

In [178]:
# protein table for our db. Same above but one protein by row
protein = pd.read_csv('db_tables/protein.tsv', sep='\t')

In [179]:
# DataFrame with unique id_protein col
id_protein = protein[['id_protein', 'uniprot_acc']].copy()

In [180]:
# only clinvar mutations at the moment
mutations = pd.read_csv('../datasets/clinvar_all_proteins_mutations.csv.gz', compression='gzip') # comes from parse_clinvar.py

In [181]:
disorder = pd.read_csv('disorder_lite.csv').rename(columns={'uniprot': 'uniprot_acc'})
low_complexity = pd.read_csv('low_complexity.csv').rename(columns={'uniprot': 'uniprot_acc'})
pfam = pd.read_csv('pfam.csv').rename(columns={'uniprot': 'uniprot_acc'})

In [182]:
# Add and unique integer ID fow low_complexity and disorder
low_complexity['id_lc'] = range(1, len(low_complexity)+1)
disorder['id_idr'] = range(1, len(disorder)+1)

---  
# consequence and source tables

In [183]:
cf = mutations.consequence.value_counts()
cf

missense       132500
frameshit       18891
nonsense        11265
deletion         2610
insertion        1103
delins            763
duplication       619
Name: consequence, dtype: int64

In [184]:
consequence = pd.DataFrame({'id_consequence': range(1, len(cf)+1), 'consequence': cf.index})
consequence

,id_consequence,consequence
0,1,missense
1,2,frameshit
2,3,nonsense
3,4,deletion
4,5,insertion
5,6,delins
6,7,duplication


In [185]:
mutations.source.value_counts()

clinvar    167751
Name: source, dtype: int64

In [186]:
source = pd.DataFrame({'id_source': [1,2,3], 'source': ['clinvar', 'disgenet', 'uniprot']})
source

,id_source,source
0,1,clinvar
1,2,disgenet
2,3,uniprot


In [187]:
consequence.to_csv('db_tables/consequence.tsv', sep='\t', index = False)

In [188]:
source.to_csv('db_tables/source.tsv', sep='\t', index = False)

---  
# mutation table  
cols: *id_mutation, snp_id, chromosome, start_genomic, end_genomic, start_aa, end_aa, from_aa, to_aa, id_source, id_protein*

In [189]:
mutations[~mutations.end_aa.isnull()][['start_aa',	'end_aa',	'from',	'to',	'consequence',	'source']]

,start_aa,end_aa,from,to,consequence,source
5,1755,1757.0,LeuThr,NaN,deletion,clinvar
13,23,24.0,GlyGlu,NaN,deletion,clinvar
14,47,54.0,GlyArg,NaN,deletion,clinvar
16,295,298.0,AspLeu,NaN,deletion,clinvar
18,116,124.0,GluVal,NaN,deletion,clinvar
...,...,...,...,...,...,...
23978,1,2.0,MetGly,Ala,insertion,clinvar
23979,517,518.0,IleLeu,Ter,insertion,clinvar
23981,229,230.0,GlyPro,NaN,insertion,clinvar
23984,1171,1172.0,AspGlu,NaN,insertion,clinvar


In [190]:
mutations.columns

Index(['uniprot_acc', 'organism', 'mlo', 'rol', 'db', 'hgnc_id', 'gene_name',
       'approved_name', 'gene_id', 'geneid', 'genesymbol', 'snpid', 'alleleid',
       'chromosomeaccession', 'chromosome', 'start', 'stop', 'type', 'name',
       'origin', 'phenotypeids', 'phenotypelist', 'otherids', 'nuccore_id',
       'cambio', 'start_aa', 'end_aa', 'from', 'to', 'consequence', 'source'],
      dtype='object')

In [191]:
# Subset by cols to keep for mutation db table
mutation = mutations[['uniprot_acc', 'snpid', 'chromosome', 'start', 'stop', 'start_aa', 'end_aa', 'from', 'to', 'consequence']].copy()
mutation.rename(columns={'snpid': 'snp_id', 'start': 'start_genomic', 'stop': 'end_genomic', 'from': 'from_aa', 'to': 'to_aa'}, inplace= True)

In [192]:
# Add an unique ID for each mutation, type INT
mutation['id_mutation'] = range(1, len(mutation)+1)

In [193]:
# Fill those mutations containing NaNs in the end_aa col with the start_aa value
mutation.end_aa = mutation.end_aa.fillna(value= mutation.start_aa).apply(int)

In [194]:
mutation.head()

,uniprot_acc,snp_id,chromosome,start_genomic,end_genomic,start_aa,end_aa,from_aa,to_aa,consequence,id_mutation
0,A6H8Y1,879255413,5,71512291,71512293,1371,1371,Arg,NaN,deletion,1
1,A6NHR9,886043345,18,2700757,2700759,497,497,Lys,NaN,deletion,2
2,A6NHR9,886044914,18,2697985,2697987,430,430,His,NaN,deletion,3
3,A6NHR9,1598342592,18,2705766,2705768,639,639,Gly,NaN,deletion,4
4,A6NHR9,1598293848,18,2673343,2673345,164,164,Arg,NaN,deletion,5


In [195]:
# Add IDs from protein and consequence
mutation = mutation.merge(id_protein)
mutation = mutation.merge(consequence)
mutation.drop(columns=['uniprot_acc', 'consequence'], inplace= True)

In [196]:
def format_snp(df, column):
    '''
    format an int snps column in a DataFrame containing -1 values.
    Returns: the snp column in str format ('rs1580653772' or 'nan')
    '''
    #a = df.column.replace(-1, 'nan')
    a = df[column]
    a = a.apply(str)
    a = a.map(lambda x: 'rs' + x)
    a = a.replace('rs-1', 'nan')
    df[column] = a

In [197]:
# Format snp_id col
format_snp(mutation, 'snp_id')

In [198]:
# Format from_aa and to_aa cols to 1 letter code
from Bio.SeqUtils import seq1
mutation['from_aa'] = mutation['from_aa'].map(lambda x: seq1(x))
mutation['to_aa'] = mutation['to_aa'].apply(str).map(lambda x: seq1(x) if x != 'nan' else x)

In [199]:
mutation.columns

Index(['snp_id', 'chromosome', 'start_genomic', 'end_genomic', 'start_aa',
       'end_aa', 'from_aa', 'to_aa', 'id_mutation', 'id_protein',
       'id_consequence'],
      dtype='object')

In [200]:
mutation = mutation[['id_mutation', 'snp_id', 'chromosome', 'start_genomic', 'end_genomic', 'start_aa','end_aa',
                    'from_aa', 'to_aa', 'id_protein', 'id_consequence']].sort_values('id_mutation')
mutation

,id_mutation,snp_id,chromosome,start_genomic,end_genomic,start_aa,end_aa,from_aa,to_aa,id_protein,id_consequence
0,1,rs879255413,5,71512291,71512293,1371,1371,R,nan,19,4
1,2,rs886043345,18,2700757,2700759,497,497,K,nan,25,4
2,3,rs886044914,18,2697985,2697987,430,430,H,nan,25,4
3,4,rs1598342592,18,2705766,2705768,639,639,G,nan,25,4
4,5,rs1598293848,18,2673343,2673345,164,164,R,nan,25,4
...,...,...,...,...,...,...,...,...,...,...,...
164603,167747,rs1064795493,2,51028155,51028155,40,40,W,*,4362,3
164604,167748,rs1553759318,2,50538311,50538311,695,695,W,*,4362,3
164605,167749,rs1201575289,2,50055006,50055006,1253,1253,R,*,4362,3
164606,167750,nan,2,49922240,49922240,1410,1410,R,*,4362,3


In [201]:
mutation.chromosome = mutation.chromosome.apply(str)

In [202]:
mutation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167751 entries, 0 to 164607
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id_mutation     167751 non-null  int32 
 1   snp_id          167751 non-null  object
 2   chromosome      167751 non-null  object
 3   start_genomic   167751 non-null  int64 
 4   end_genomic     167751 non-null  int64 
 5   start_aa        167751 non-null  int64 
 6   end_aa          167751 non-null  int64 
 7   from_aa         167751 non-null  object
 8   to_aa           167751 non-null  object
 9   id_protein      167751 non-null  int64 
 10  id_consequence  167751 non-null  int64 
dtypes: int32(1), int64(6), object(4)
memory usage: 14.7+ MB


In [203]:
mutation.to_csv('db_tables/mutation.tsv', sep='\t', index = False)

---
## Para asignar los rangos debo tener:  
- Tabla de mutaciones con id_mutation, *id_protein(Chromosome), start_aa(Start), end_aa(End)*  
- Tablas de lc, idr y pfam con id unico 

---  
# Pfam Tables

In [204]:
pfam.head()

,uniprot_acc,tipo,start,end
0,O94910,7tm_2,857,1093
1,Q9HAR2,7tm_2,861,1097
2,O14514,7tm_2,944,1180
3,O75899,7tm_3,475,743
4,Q9NZH0,7tm_3,49,291


## pfam_domain  
cols: pfam_id, pfam_domain, por ej: PF00003 7tm_3

In [205]:
# Array with unique pfam domains
pf_domain = pfam.tipo.unique() # unique pfam domains (2939 for this set of proteins)

In [206]:
pfam_domain = pd.DataFrame({'pfam_domain': pf_domain, 'id_pfam': range(1, len(pf_domain)+1)})  # luego cambiar el id por los PF000...
pfam_domain

,pfam_domain,id_pfam
0,7tm_2,1
1,7tm_3,2
2,ATP-synt_ab,3
3,GTP_EFTU,4
4,HLH,5
...,...,...
2934,PhoLip_ATPase_C,2935
2935,HIP1_clath_bdg,2936
2936,DAO_C,2937
2937,Armet,2938


In [207]:
pfam_domain.to_csv('db_tables/pfam_domain.tsv', sep='\t', index= False)

## protein_has_pfam_domain  
cols: id_protein, id_pfam, start, end, length

In [208]:
protein_has_pfam_domain = pfam.merge(id_protein) # agregar col id_protein
protein_has_pfam_domain['length'] = protein_has_pfam_domain.end - protein_has_pfam_domain.start + 1 # col length

In [209]:
protein_has_pfam_domain.rename(columns= {'tipo': 'pfam_domain'}, inplace= True)
protein_has_pfam_domain = protein_has_pfam_domain.merge(pfam_domain) # to add the col pfam_id

In [210]:
protein_has_pfam_domain = protein_has_pfam_domain[['id_protein', 'id_pfam', 'start', 'end', 'length']].sort_values('id_protein')
protein_has_pfam_domain

,id_protein,id_pfam,start,end,length
6140,1,171,17,144,128
9114,2,814,28,91,64
627,4,50,153,220,68
626,4,50,248,312,65
625,4,50,73,141,69
...,...,...,...,...,...
1461,4365,784,55,184,130
1457,4365,784,475,619,145
5600,4367,2244,160,199,40
10551,4367,2529,305,369,65


In [211]:
protein_has_pfam_domain.to_csv('db_tables/protein_has_pfam_domain.tsv', sep='\t', index= False)

## mutation_has_pfam_domain  
cols: id_mutation, id_protein, id_pfam, start, end

### Pyranges  
columnas obligatorias: *Chromosome	 Start	End*  
Chromosome: id_protein    
otras columnas con ids son opcionales y cualquier nombre  
  
por ejemplo df seria tabla de mutaciones  
df = pr.PyRanges(df.rename(columns={'chromosome':'Chromosome','start_position':'Start','end_position':'End'}))  
  
df = pyrange de mutaciones (columnas: Chromosome, Start, End, id_mutacion)  
low_c = pyrange de low complexity(columnas: Chromosome, Start, End, id_low, id_proteina)  
data = df.join(low_c, strandedness=False, slack=1).drop(like="_b") # mutaciones lo junto con low_complex  
strandedness=False no tener en cuenta el Strand  
slack=1 coincidir los extremos. Importante  
drop(like="_b") eliminar el Chromosome, Start, End de low_c (en pfam no hacer el drop)  
data = data.df[[Chromosome, Start, End, id_mutacion, id_low, id_proteina]] # pasa de pyrange a dataframe

In [212]:
mutation.head()

,id_mutation,snp_id,chromosome,start_genomic,end_genomic,start_aa,end_aa,from_aa,to_aa,id_protein,id_consequence
0,1,rs879255413,5,71512291,71512293,1371,1371,R,nan,19,4
1,2,rs886043345,18,2700757,2700759,497,497,K,nan,25,4
2,3,rs886044914,18,2697985,2697987,430,430,H,nan,25,4
3,4,rs1598342592,18,2705766,2705768,639,639,G,nan,25,4
4,5,rs1598293848,18,2673343,2673345,164,164,R,nan,25,4


In [213]:
# df has pfam domains data
df = pfam.rename(columns={'tipo': 'pfam_domain'}).merge(pfam_domain)
df = df.merge(id_protein)                      # mapping uniprot_acc - id_protein
df.drop(columns='uniprot_acc', inplace= True)

In [214]:
df.head()

,pfam_domain,start,end,id_pfam,id_protein
0,7tm_2,857,1093,1,477
1,GPS,800,844,690,477
2,Gal_Lectin,48,128,745,477
3,OLF,144,396,773,477
4,Latrophilin,1113,1474,818,477


In [215]:
df.rename(columns={'id_protein': 'Chromosome', 'start': 'Start', 'end': 'End'}, inplace= True)

In [216]:
df.head()

,pfam_domain,Start,End,id_pfam,Chromosome
0,7tm_2,857,1093,1,477
1,GPS,800,844,690,477
2,Gal_Lectin,48,128,745,477
3,OLF,144,396,773,477
4,Latrophilin,1113,1474,818,477


In [217]:
# Create the pyranges object of pfam domains
df_py = pr.PyRanges(df)

In [218]:
aux = mutation[['start_aa', 'end_aa', 'id_mutation', 'id_protein']].copy()
aux.rename(columns={'id_protein': 'Chromosome', 'start_aa': 'Start', 'end_aa': 'End'}, inplace= True)

In [219]:
aux.head()

,Start,End,id_mutation,Chromosome
0,1371,1371,1,19
1,497,497,2,25
2,430,430,3,25
3,639,639,4,25
4,164,164,5,25


In [220]:
# Pyranges object of mutations
aux_py = pr.PyRanges(aux)

In [221]:
# Join both pyranges object: this assings mutations to pfam domains
pfam_py = df_py.join(aux_py, strandedness= False, slack= 1)  # strandedness= False doesnt take count of the chain strand; slack= 1 include bounds

In [222]:
pfam_py.head() # Start and End are from the pfam domain in that protein (a protein may have the same pfam domain repeated at different positions along its sequence).
                # Start_b and End_b are from the mutation in this case

,pfam_domain,Start,End,id_pfam,Chromosome,Start_b,End_b,id_mutation
0,UCR_hinge,28,91,814,2,53,53,23987
1,An_peroxidase,727,1272,947,9,981,981,23988
2,An_peroxidase,727,1272,947,9,1039,1039,23994
3,An_peroxidase,727,1272,947,9,1133,1133,23989
4,An_peroxidase,727,1272,947,9,1207,1207,23993
5,LRR_8,50,110,2219,9,65,65,23996
6,Ig_3,329,402,2246,9,391,391,24000
7,I-set,511,597,2914,9,538,538,23997


In [223]:
# Pyranges to DataFrame
mutation_has_pfam_domain = pfam_py.df[['id_mutation', 'Chromosome', 'id_pfam', 'Start', 'End']] # cols to keep

In [224]:
mutation_has_pfam_domain.rename(columns={'Chromosome': 'id_protein', 'Start': 'start', 'End': 'end'}, inplace= True)

In [225]:
mutation_has_pfam_domain.head()

,id_mutation,id_protein,id_pfam,start,end
0,23987,2,814,28,91
1,23988,9,947,727,1272
2,23994,9,947,727,1272
3,23989,9,947,727,1272
4,23993,9,947,727,1272


In [226]:
# control
mutation[mutation.id_mutation == 23987]

,id_mutation,snp_id,chromosome,start_genomic,end_genomic,start_aa,end_aa,from_aa,to_aa,id_protein,id_consequence
148020,23987,rs7417535,1,15807492,15807492,53,53,Y,C,2,1


In [227]:
# control
pfam_domain[pfam_domain.id_pfam == 814] 

,pfam_domain,id_pfam
813,UCR_hinge,814


In [228]:
mutation_has_pfam_domain[mutation_has_pfam_domain.id_pfam == 814] # ok!

,id_mutation,id_protein,id_pfam,start,end
0,23987,2,814,28,91


In [229]:
mutation_has_pfam_domain.to_csv('db_tables/mutation_has_pfam_domain.tsv', sep='\t', index= False)

---  
# low-complexity Tables

## low_complexity  
cols: id_lc, start, end, length, id_protein

In [230]:
low_complexity.head()

,uniprot_acc,start,end,id_lc
0,P61981,236,243,1
1,P31947,235,247,2
2,P31947,248,247,3
3,P27348,230,244,4
4,P27348,245,244,5


In [231]:
# Add length col 
low_complexity['length'] = low_complexity.end - low_complexity.start + 1 

In [232]:
# Add id_proteins
low_complexity.rename(columns={'uniprot': 'uniprot_acc'}, inplace= True)
low_complexity = low_complexity.merge(id_protein)
low_complexity.drop(columns='uniprot_acc', inplace= True)

In [233]:
low_complexity.head()

,start,end,id_lc,length,id_protein
0,236,243,1,8,1602
1,235,247,2,13,1132
2,248,247,3,0,1132
3,230,244,4,15,1049
4,245,244,5,0,1049


In [234]:
low_complexity.to_csv('db_tables/low_complexity.tsv', sep='\t', index= False)

## mutation_has_low_complexity  
cols: id_mutation, id_lc

In [235]:
# Table for LC data
lc_has = low_complexity.copy()
lc_has.rename(columns={'id_protein': 'Chromosome', 'start': 'Start', 'end': 'End'}, inplace= True)

In [236]:
# Auxiliar table for mutations
aux_lc = mutation[['start_aa', 'end_aa', 'id_mutation', 'id_protein']].copy()
aux_lc.rename(columns={'id_protein': 'Chromosome', 'start_aa': 'Start', 'end_aa': 'End'}, inplace= True)

In [237]:
aux_lc.head()

,Start,End,id_mutation,Chromosome
0,1371,1371,1,19
1,497,497,2,25
2,430,430,3,25
3,639,639,4,25
4,164,164,5,25


In [238]:
# Create the Pyranges objects
lc_has_py = pr.PyRanges(lc_has)
aux_lc_py = pr.PyRanges(aux_lc)

In [239]:
# Join both pyranges object: this assings mutations to low-complexity regions
lc_py = aux_lc_py.join(lc_has_py, strandedness= False, slack=1).drop(like="_b") # strandedness= False doesnt take count of the chain strand;
                                                                                # slack= 1 include bounds; drop(like="_b"): delete those cols (redudants)

In [240]:
lc_py.head()

,Start,End,id_mutation,Chromosome,id_lc,length
0,1133,1133,23989,9,5240,12
1,287,287,3994,16,9733,28
2,197,197,3995,16,9732,29
3,336,336,24003,17,5023,26
4,1038,1038,24009,18,11903,20
5,1469,1469,24030,19,10455,18
6,5,5,24048,23,5786,14
7,7,7,24056,23,5786,14


In [241]:
# Pyrange to DataFrame
mutation_has_low_complexity = lc_py.df[['id_mutation', 'id_lc']] # cols to keep

In [242]:
mutation_has_low_complexity.head()

,id_mutation,id_lc
0,23989,5240
1,3994,9733
2,3995,9732
3,24003,5023
4,24009,11903


In [243]:
# Control
low_complexity[low_complexity.id_lc == 5240]

,start,end,id_lc,length,id_protein
5239,1128,1139,5240,12,9


In [244]:
protein.iloc[8]

id_protein                                                          9
uniprot_acc                                                    A1KZ92
hgnc_id                                                    HGNC:26359
gene_id                                                      137902.0
gene_name                                                       PXDNL
length                                                           1463
sequence            MEPRLFCWTTLFLLAGWCLPGLPCPSRCLCFKSTVRCMHLMLDHIP...
disorder_content                                                  NaN
Name: 8, dtype: object

In [245]:
mutation[mutation.id_mutation == 23989] # It's allright! Mutation in aa 1133, which belongs to the low-complexity region between 1128 - 1139 in that protein

,id_mutation,snp_id,chromosome,start_genomic,end_genomic,start_aa,end_aa,from_aa,to_aa,id_protein,id_consequence
138298,23989,rs74731075,8,51408226,51408226,1133,1133,A,V,9,1


In [246]:
mutation_has_low_complexity.to_csv('db_tables/mutation_has_low_complexity.tsv', sep='\t', index= False)

---  
# Disorder Tables

## disorder_region  
cols: id_idr, start, end, length, id_protein

In [247]:
# Add length col 
disorder['length'] = disorder.end - disorder.start + 1 

In [248]:
disorder_region = disorder.rename(columns={'uniprot': 'uniprot_acc'}).merge(id_protein).sort_values('id_protein')
disorder_region.drop(columns='uniprot_acc', inplace= True)
disorder_region.head()

,start,end,id_idr,length,id_protein
2338,1,30,2339,30,2
1426,1,68,1427,68,3
2489,1,25,2490,25,4
5782,660,754,5783,95,5
5781,1,103,5782,103,5


In [249]:
disorder_region.to_csv('db_tables/disorder_region.tsv', sep='\t', index= False)

## mutation_has_disorder_region  
cols: id_mutation, id_idr

In [250]:
# Auxiliar table for mutations from low-complexity is the same for disorder. id-protein, start and end of the mutation
aux_idr = aux_lc
aux_idr.head()

,Start,End,id_mutation,Chromosome
0,1371,1371,1,19
1,497,497,2,25
2,430,430,3,25
3,639,639,4,25
4,164,164,5,25


In [251]:
# Table for IDRs data
idr_has = disorder_region.copy()
idr_has.rename(columns={'id_protein': 'Chromosome', 'start': 'Start', 'end': 'End'}, inplace= True)
idr_has.head()

,Start,End,id_idr,length,Chromosome
2338,1,30,2339,30,2
1426,1,68,1427,68,3
2489,1,25,2490,25,4
5782,660,754,5783,95,5
5781,1,103,5782,103,5


In [252]:
# Create the Pyranges objects
idr_has_py = pr.PyRanges(idr_has)
aux_idr_py = pr.PyRanges(aux_idr)

In [253]:
# Join both pyranges object: this assings mutations to pfam domains
idr_py = aux_idr_py.join(idr_has_py, strandedness= False, slack=1).drop(like="_b") # strandedness= False doesnt take count of the chain strand;
                                                                                   # slack= 1 include bounds; drop(like="_b"): delete those cols (redudants)

In [254]:
idr_py.head()

,Start,End,id_mutation,Chromosome,id_idr,length
0,70,70,24001,12,4339,28
1,287,287,3994,16,4224,129
2,336,336,24003,17,2225,157
3,1014,1014,24004,17,2227,49
4,1371,1371,1,19,4637,76
5,1180,1180,24014,19,4635,293
6,2580,2580,24016,19,4645,65
7,213,213,24021,19,4630,49


In [255]:
# Pyrange to DataFrame
mutation_has_disorder_region = idr_py.df[['id_mutation', 'id_idr']] # cols to keep
mutation_has_disorder_region.head()

,id_mutation,id_idr
0,24001,4339
1,3994,4224
2,24003,2225
3,24004,2227
4,1,4637


In [256]:
# Control
mutation[mutation.id_mutation == 24001]

,id_mutation,snp_id,chromosome,start_genomic,end_genomic,start_aa,end_aa,from_aa,to_aa,id_protein,id_consequence
148021,24001,rs116340837,6,149474335,149474335,70,70,A,V,12,1


In [257]:
id_protein[id_protein.id_protein == 12]

,id_protein,uniprot_acc
11,12,A2A288


In [258]:
disorder[disorder.id_idr == 4339] # It's Ok. A point mutation in position 70 in the idr region between 48-75

,uniprot_acc,start,end,id_idr,length
4338,A2A288,48,75,4339,28


In [259]:
mutation_has_disorder_region.to_csv('db_tables/mutation_has_disorder_region.tsv', sep='\t', index= False)

---   
# Rol table  
cols: id_rol, rol

In [261]:
entrada_dbs.rol.unique()

array(['driver', 'component', 'regulator', 'client'], dtype=object)

In [262]:
entrada_dbs.rol.value_counts()

client       5654
regulator    1395
component     836
driver        381
Name: rol, dtype: int64

In [13]:
rol = pd.DataFrame({'rol': entrada_dbs.rol.value_counts().index, 'id_rol': range(1, len(entrada_dbs.rol.value_counts())+1)})
rol

,rol,id_rol
0,client,1
1,regulator,2
2,component,3
3,driver,4


In [263]:
rol.to_csv('db_tables/rol.tsv', sep='\t', index= False)

---  
# database table  
cols: id_database, database

In [264]:
entrada_dbs.db.value_counts()

drllps              5034
phasepdb_ht         2426
phasepdb_uniprot     384
phasepdb_rev         297
phasepro             125
Name: db, dtype: int64

In [265]:
database = pd.DataFrame({'database': entrada_dbs.db.value_counts().index, 'id_database': range(1, len(entrada_dbs.db.value_counts())+1)})
database

,database,id_database
0,drllps,1
1,phasepdb_ht,2
2,phasepdb_uniprot,3
3,phasepdb_rev,4
4,phasepro,5


In [266]:
database.to_csv('db_tables/database.tsv', sep='\t', index= False)

---  
# MLOs tables  

In [267]:
entrada_dbs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8266 entries, 0 to 8391
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniprot_acc  8266 non-null   object
 1   mlo          8266 non-null   object
 2   rol          8266 non-null   object
 3   db           8266 non-null   object
dtypes: object(4)
memory usage: 322.9+ KB


In [268]:
entrada_dbs.mlo.value_counts()

Nucleolus                      1972
Stress granule                  918
Postsynaptic density            917
null_phasepdb_ht                750
P-body                          698
                               ... 
Cytoplasmic granule               1
 cGAS foci                        1
 granular component               1
heterochromatin                   1
 liquid-like DYRK3 speckles       1
Name: mlo, Length: 148, dtype: int64

In [269]:
len(entrada_dbs.uniprot_acc.unique())

4368

In [270]:
len(entrada_dbs.mlo.unique()) # there's blank spaces; unify data

148

## Deal with mlos annotations

In [271]:
entrada_dbs.mlo = entrada_dbs.mlo.str.strip()

In [272]:
entrada_dbs.mlo.value_counts()

Nucleolus                        2063
Stress granule                   1407
Postsynaptic density             1374
P-body                            830
null_phasepdb_ht                  750
                                 ... 
galectin lattice                    1
Cytoplasmic granule                 1
inclusion body                      1
RNA polymerase II, holoenzyme       1
granular component                  1
Name: mlo, Length: 116, dtype: int64

### Paraspeckle

In [273]:
(entrada_dbs.mlo == 'Paraspeckle').sum()

101

In [274]:
# Unify paraspeckle with Paraspeckle
entrada_dbs.replace('paraspeckle', 'Paraspeckle', inplace= True)

In [275]:
(entrada_dbs.mlo == 'Paraspeckle').sum()

104

### Sam68

In [276]:
(entrada_dbs.mlo == 'Sam68 nuclear bodies').sum()

11

In [277]:
(entrada_dbs.mlo == 'Sam68 nuclear bodies (SNBs)').sum()

2

In [278]:
(entrada_dbs.mlo == 'Sam68 nuclear body').sum()

12

In [279]:
entrada_dbs.replace(['Sam68 nuclear bodies', 'Sam68 nuclear bodies (SNBs)'], 'Sam68 nuclear body', inplace= True)
(entrada_dbs.mlo == 'Sam68 nuclear body').sum()

25

### PML body  
**PhaSepDB**: The PML bodies are dynamic nuclear protein aggregates interspersed between chromatin. These punctate nuclear structures are call PML bodies because the PML gene is essential for their formation. are present in most mammalian cell nuclei and typically number 1 to 30 bodies per nucleus.  
**DrLLPS**: PML nuclear bodies are annotetad in the nucleus. They are matrix-associated domains that recruit an astonishing variety of seemingly unrelated proteins.

In [280]:
(entrada_dbs.mlo == 'PML nuclear body').sum()

97

In [281]:
(entrada_dbs.mlo == 'PML body').sum()

77

In [282]:
entrada_dbs.replace('PML body', 'PML nuclear body', inplace= True)
(entrada_dbs.mlo == 'PML nuclear body').sum()

174

### Polycomb body

In [283]:
(entrada_dbs.mlo == 'Polycomb bodies').sum()

2

In [284]:
entrada_dbs.replace('Polycomb bodies', 'Polycomb body', inplace= True)
(entrada_dbs.mlo == 'Polycomb body').sum()

4

### Pre and postsynaptic density

In [285]:
entrada_dbs.replace('Pre and postsynaptic densities', 'Pre and postsynaptic density', inplace= True)


In [286]:
(entrada_dbs.mlo == 'Pre and postsynaptic density').sum()

8

### Nuclear speckle

In [287]:
(entrada_dbs.mlo == 'Nucleus speckles').sum() #phasepdb

115

In [288]:
(entrada_dbs.mlo == 'Nuclear speckle').sum() # drllps

110

In [289]:
(entrada_dbs.mlo == 'Nuclear speckles').sum() #phasepdb

24

In [290]:
(entrada_dbs.mlo == 'nuclear speckle').sum()

3

In [291]:
entrada_dbs.replace(['Nucleus speckles', 'Nuclear speckles', 'nuclear speckle'], 'Nuclear speckle', inplace= True)
(entrada_dbs.mlo == 'Nuclear speckle').sum()

252

### Heterochromatin

In [292]:
(entrada_dbs.mlo == 'heterochromatin').sum()

2

In [293]:
entrada_dbs.replace('heterochromatin', 'Heterochromatin', inplace= True)
(entrada_dbs.mlo == 'Heterochromatin').sum()

3

### Cytoplasmic ribonucleoprotein granule

In [294]:
(entrada_dbs.mlo == 'cytoplasmic ribonucleoprotein granule').sum()


6

In [295]:
entrada_dbs.replace('cytoplasmic ribonucleoprotein granule', 'Cytoplasmic ribonucleoprotein granule', inplace= True)

### Membrane cluster

In [296]:
(entrada_dbs.mlo == 'Membrane clusters').sum()

4

In [297]:
(entrada_dbs.mlo == 'membrane cluster').sum()

3

In [298]:
entrada_dbs.replace(['Membrane clusters', 'membrane cluster'], 'Membrane cluster', inplace= True)
(entrada_dbs.mlo == 'Membrane cluster').sum()

7

### Nuclear body

In [299]:
entrada_dbs.replace('nuclear body', 'Nuclear body', inplace= True)
(entrada_dbs.mlo == 'Nuclear body').sum()

11

### Nucleolus

In [300]:
entrada_dbs.replace('nucleolus', 'Nucleolus', inplace= True)
(entrada_dbs.mlo == 'Nucleolus').sum()

2064

In [302]:
(entrada_dbs.mlo == 'Centrosome/Spindle pole body').sum() # keep this annotation

534

## OK, now mlo table  
cols: id_mlo, mlo

In [303]:
entrada_dbs.mlo.value_counts()

Nucleolus                     2064
Stress granule                1407
Postsynaptic density          1374
P-body                         830
null_phasepdb_ht               750
                              ... 
PcG protein complex              1
liquid-like DYRK3 speckles       1
Perinucleolar compartment        1
Histone Locus body               1
granular component               1
Name: mlo, Length: 102, dtype: int64

In [305]:
# EXPLODE:
# P-body, Stress granule
# P-body, GW body
# Set mlo col as list-like and explode() to separate list elements into separate rows
# before: 8266 rows
entrada_dbs = entrada_dbs.assign(mlo= entrada_dbs.mlo.str.split(',')).explode('mlo')
entrada_dbs.mlo = entrada_dbs.mlo.str.strip()
entrada_dbs
# after: 8272 rows

,uniprot_acc,mlo,rol,db
0,P35637,cytoplasmic stress granule,driver,phasepro
1,P35637,Cytoplasmic ribonucleoprotein granule,driver,phasepro
2,Q06787,cytoplasmic stress granule,driver,phasepro
3,Q06787,Cytoplasmic ribonucleoprotein granule,driver,phasepro
4,Q06787,synaptosome,driver,phasepro
...,...,...,...,...
8381,P33778,Nucleolus,client,drllps
8384,Q13148,Cytoplasmic ribonucleoprotein granule,driver,phasepro
8387,O43561,LAT signalosome,driver,phasepro
8389,P62993,LAT signalosome,driver,phasepro


In [306]:
# GW-body
entrada_dbs.replace('GW body', 'GW-body', inplace= True)
(entrada_dbs.mlo == 'GW-body').sum()

3

In [307]:
# Postsynaptic density
entrada_dbs.replace('postsynaptic density', 'Postsynaptic density', inplace= True)
(entrada_dbs.mlo == 'Postsynaptic density').sum()

1375

In [309]:
# Cytoplasmic ribonucleoprotein granule
entrada_dbs.replace('cytoplasmic ribonucleoprotein granule', 'Cytoplasmic ribonucleoprotein granule', inplace= True)
(entrada_dbs.mlo == 'Cytoplasmic ribonucleoprotein granule').sum()

7

In [310]:
# Histone locus body
entrada_dbs.replace('Histone Locus body', 'Histone locus body', inplace= True)
(entrada_dbs.mlo == 'Histone locus body').sum()

16

In [311]:
# Stress granule
entrada_dbs.replace('Sress granule', 'Stress granule', inplace= True)
(entrada_dbs.mlo == 'Stress granule').sum()

1411

In [312]:
entrada_dbs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8272 entries, 0 to 8391
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniprot_acc  8272 non-null   object
 1   mlo          8272 non-null   object
 2   rol          8272 non-null   object
 3   db           8272 non-null   object
dtypes: object(4)
memory usage: 323.1+ KB


In [313]:
entrada_dbs.mlo.value_counts()

Nucleolus                                                                                     2064
Stress granule                                                                                1411
Postsynaptic density                                                                          1375
P-body                                                                                         834
null_phasepdb_ht                                                                               750
                                                                                              ... 
condensed compartments of microtubule bundling                                                   1
TIS granule                                                                                      1
Nuage                                                                                            1
PcG protein complex                                                                              1
selective 

In [314]:
mlo = pd.DataFrame({'mlo': entrada_dbs.mlo.value_counts().index, 'id_mlo': range(1, len(entrada_dbs.mlo.unique())+1)})
mlo

,mlo,id_mlo
0,Nucleolus,1
1,Stress granule,2
2,Postsynaptic density,3
3,P-body,4
4,null_phasepdb_ht,5
...,...,...
94,condensed compartments of microtubule bundling,95
95,TIS granule,96
96,Nuage,97
97,PcG protein complex,98


In [315]:
mlo.to_csv('db_tables/mlo.tsv', sep='\t', index= False)

## protein_has_mlo  
cols: id_protein, id_mlo, id_rol, id_database

In [316]:
len(entrada_dbs.uniprot_acc.unique())

4368

In [317]:
protein_has_mlo = entrada_dbs.copy()

In [318]:
protein_has_mlo.head()

,uniprot_acc,mlo,rol,db
0,P35637,cytoplasmic stress granule,driver,phasepro
1,P35637,Cytoplasmic ribonucleoprotein granule,driver,phasepro
2,Q06787,cytoplasmic stress granule,driver,phasepro
3,Q06787,Cytoplasmic ribonucleoprotein granule,driver,phasepro
4,Q06787,synaptosome,driver,phasepro


In [319]:
# Add id_protein
protein_has_mlo = protein_has_mlo.merge(id_protein)
# Add id_mlo
protein_has_mlo = protein_has_mlo.merge(mlo)
# Add id_rol and id_database
protein_has_mlo = protein_has_mlo.merge(rol)
protein_has_mlo = protein_has_mlo.rename(columns={'db': 'database'}).merge(database).sort_values('id_protein')

In [320]:
protein_has_mlo.drop(columns=['uniprot_acc', 'mlo', 'rol', 'database'], inplace= True)
protein_has_mlo

,id_protein,id_mlo,id_rol,id_database
8271,1,5,2,2
4709,2,2,2,1
1082,3,4,1,1
5185,3,4,3,3
1276,4,4,1,1
...,...,...,...,...
1310,4366,4,1,1
4447,4367,20,1,1
406,4368,10,1,1
2237,4368,1,1,1


In [321]:
protein_has_mlo.to_csv('db_tables/protein_has_mlo.tsv', sep='\t', index= False)

In [131]:
#aa = entrada_dbs[['uniprot_acc', 'rol']].drop_duplicates().groupby('uniprot_acc').size().reset_index(name='counts')

In [134]:
#aa['counts'].unique().tolist()

[1, 2, 3, 4]

In [135]:
#aa[aa.counts == 4].head()

,uniprot_acc,counts
733,P06748,4
1160,P29590,4
1196,P31483,4
1286,P38432,4
1353,P43243,4


In [139]:
#entrada_dbs[entrada_dbs.uniprot_acc == 'P06748'] # es el rol en la db

,uniprot_acc,organism,mlo,rol,db
32,P06748,Homo sapiens,Nucleolus,driver,phasepro
33,P06748,Homo sapiens,granular component,driver,phasepro
177,P06748,Homo sapiens,Nucleolus,component,phasepdb_rev
401,P06748,Homo sapiens,null_phasepdb_rev,component,phasepdb_rev
1384,P06748,Homo sapiens,Nucleolus,client,phasepdb_ht
1696,P06748,Homo sapiens,Nucleolus,client,phasepdb_ht
2398,P06748,Homo sapiens,Stress granule,client,phasepdb_ht
3076,P06748,Homo sapiens,null_phasepdb_ht,regulator,phasepdb_ht
3386,P06748,Homo sapiens,Droplet,driver,drllps
3387,P06748,Homo sapiens,Nucleolus,driver,drllps
